<a href="https://colab.research.google.com/github/denys-khvashchenko/AB_Test_Statistical_Analisis-Python/blob/main/ab_test_statistical_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import auth
from google.cloud import bigquery

import numpy as np
import pandas as pd
from scipy.stats import chi2_contingency
from statsmodels.stats.proportion import proportions_ztest

auth.authenticate_user()
client = bigquery.Client(project="data-analytics-mate")

query = """
WITH
  session_info AS (
  SELECT
    session.date,
    session.ga_session_id,
    session_params.country,
    session_params.device,
    session_params.continent,
    session_params.channel,
    ab.test,
    ab.test_group
  FROM
    `DA.ab_test` AS ab
  JOIN
    `DA.session` AS session
  ON
    ab.ga_session_id = session.ga_session_id
  JOIN
    `DA.session_params` AS session_params
  ON
    session.ga_session_id = session_params.ga_session_id ),
  session_with_orders AS (
  SELECT
    session_info.date,
    session_info.country,
    session_info.device,
    session_info.continent,
    session_info.channel,
    session_info.test,
    session_info.test_group,
    COUNT(DISTINCT orders.ga_session_id) AS session_with_orders
  FROM
    `DA.order` AS orders
  JOIN
    session_info
  ON
    orders.ga_session_id = session_info.ga_session_id
  GROUP BY
    session_info.date,
    session_info.country,
    session_info.device,
    session_info.continent,
    session_info.channel,
    session_info.test,
    session_info.test_group ),
  events AS (
  SELECT
    session_info.date,
    session_info.country,
    session_info.device,
    session_info.continent,
    session_info.channel,
    session_info.test,
    session_info.test_group,
    event_params.event_name,
    COUNT(event_params.ga_session_id) AS event_cnt
  FROM
    `DA.event_params` AS event_params
  JOIN
    session_info
  ON
    event_params.ga_session_id = session_info.ga_session_id
  GROUP BY
    session_info.date,
    session_info.country,
    session_info.device,
    session_info.continent,
    session_info.channel,
    session_info.test,
    session_info.test_group,
    event_params.event_name ),
  session AS (
  SELECT
    session_info.date,
    session_info.country,
    session_info.device,
    session_info.continent,
    session_info.channel,
    session_info.test,
    session_info.test_group,
    COUNT(DISTINCT session_info.ga_session_id) AS session_cnt
  FROM
    session_info
  GROUP BY
    session_info.date,
    session_info.country,
    session_info.device,
    session_info.continent,
    session_info.channel,
    session_info.test,
    session_info.test_group ),
  account AS (
  SELECT
    session_info.date,
    session_info.country,
    session_info.device,
    session_info.continent,
    session_info.channel,
    session_info.test,
    session_info.test_group,
    COUNT(DISTINCT account_session.ga_session_id) AS new_accounts_cnt
  FROM
    `DA.account_session` AS account_session
  JOIN
    session_info
  ON
    account_session.ga_session_id = session_info.ga_session_id
  GROUP BY
    session_info.date,
    session_info.country,
    session_info.device,
    session_info.continent,
    session_info.channel,
    session_info.test,
    session_info.test_group )
SELECT
  session_with_orders.date,
  session_with_orders.country,
  session_with_orders.device,
  session_with_orders.continent,
  session_with_orders.channel,
  session_with_orders.test,
  session_with_orders.test_group,
  'session_with_orders' AS event_name,
  session_with_orders.session_with_orders AS value
FROM
  session_with_orders
UNION ALL
SELECT
  events.date,
  events.country,
  events.device,
  events.continent,
  events.channel,
  events.test,
  events.test_group,
  events.event_name,
  events.event_cnt AS value
FROM
  events
UNION ALL
SELECT
  session.date,
  session.country,
  session.device,
  session.continent,
  session.channel,
  session.test,
  session.test_group,
  'session' AS event_name,
  session_cnt AS value
FROM
  session
UNION ALL
SELECT
  account.date,
  account.country,
  account.device,
  account.continent,
  account.channel,
  account.test,
  account.test_group,
  'new account' AS event_name,
  new_accounts_cnt AS value
FROM
  account
"""

query_db = client.query(query)
results = query_db.result()
df = results.to_dataframe()

In [ ]:
df.head()

,date,country,device,continent,channel,test,test_group,event_name,value
0,2020-11-01,Lithuania,mobile,Europe,Organic Search,2,2,new account,1
1,2020-11-01,El Salvador,desktop,Americas,Social Search,2,1,new account,1
2,2020-11-01,Slovakia,mobile,Europe,Paid Search,2,2,new account,1
3,2020-11-01,Lithuania,desktop,Europe,Paid Search,2,2,new account,1
4,2020-11-02,North Macedonia,desktop,Europe,Direct,2,1,new account,1


In [ ]:
def analyze_ab_test(df):
    results = []
    unique_tests = df['test'].unique()
    unique_metrics = [
        'new account', 'session_with_orders', 'session', 'begin_checkout',
        'add_shipping_info', 'add_to_cart', 'add_payment_info'
    ]

    for test in unique_tests:
        test_df = df[df['test'] == test]
        for metric in unique_metrics:
            if metric == 'session':
                continue

            numerator_event = metric
            denominator_event = 'session'

            group1_df = test_df[test_df['test_group'] == 1]
            numerator_count_group1 = group1_df[group1_df['event_name'] == numerator_event]['value'].sum()
            denominator_count_group1 = group1_df[group1_df['event_name'] == denominator_event]['value'].sum()
            conversion_rate_group1 = (numerator_count_group1 / denominator_count_group1) if denominator_count_group1 > 0 else 0

            group2_df = test_df[test_df['test_group'] == 2]
            numerator_count_group2 = group2_df[group2_df['event_name'] == numerator_event]['value'].sum()
            denominator_count_group2 = group2_df[group2_df['event_name'] == denominator_event]['value'].sum()
            conversion_rate_group2 = (numerator_count_group2 / denominator_count_group2) if denominator_count_group2 > 0 else 0

            metric_change = ((conversion_rate_group2 - conversion_rate_group1) / conversion_rate_group1 * 100) if conversion_rate_group1 != 0 else np.inf

            if denominator_count_group1 > 0 and denominator_count_group2 > 0:
                count = np.array([numerator_count_group1, numerator_count_group2])
                nobs = np.array([denominator_count_group1, denominator_count_group2])
                z_stat, p_value = proportions_ztest(count, nobs, alternative='two-sided')
                significant = p_value < 0.05
            else:
                z_stat = np.nan
                p_value = np.nan
                significant = False

            results.append({
                'test': test,
                'metric': metric,
                'numerator_event': numerator_event,
                'denominator_event': denominator_event,
                'numerator_count_group1': numerator_count_group1,
                'denominator_count_group1': denominator_count_group1,
                'conversion_rate_group1': conversion_rate_group1,
                'numerator_count_group2': numerator_count_group2,
                'denominator_count_group2': denominator_count_group2,
                'conversion_rate_group2': conversion_rate_group2,
                'metric_change': metric_change,
                'z_stat': z_stat,
                'p_value': p_value,
                'significant': significant
            })

    return pd.DataFrame(results)

pivot_table = analyze_ab_test(df)

pivot_table

,test,metric,numerator_event,denominator_event,numerator_count_group1,denominator_count_group1,conversion_rate_group1,numerator_count_group2,denominator_count_group2,conversion_rate_group2,metric_change,z_stat,p_value,significant
0,2,new account,new account,session,4165,50637,0.082252,4184,50244,0.083274,1.241934,-0.588793,0.556000,False
1,2,session_with_orders,session_with_orders,session,5102,50637,0.100756,5003,50244,0.099574,-1.173410,0.625388,0.531717,False
2,2,begin_checkout,begin_checkout,session,4262,50637,0.084168,4313,50244,0.085841,1.988164,-0.952898,0.340642,False
3,2,add_shipping_info,add_shipping_info,session,3480,50637,0.068724,3510,50244,0.069859,1.650995,-0.709557,0.477979,False
4,2,add_to_cart,add_to_cart,session,2811,50637,0.055513,3061,50244,0.060923,9.745380,-3.669417,0.000243,True
5,2,add_payment_info,add_payment_info,session,2344,50637,0.046290,2409,50244,0.047946,3.576911,-1.240994,0.214608,False
6,1,new account,new account,session,3823,45362,0.084278,3681,45193,0.081451,-3.354299,1.542883,0.122859,False
7,1,session_with_orders,session_with_orders,session,4514,45362,0.099511,4526,45193,0.100148,0.640785,-0.320049,0.748931,False
8,1,begin_checkout,begin_checkout,session,3784,45362,0.083418,4021,45193,0.088974,6.660587,-2.978783,0.002894,True
9,1,add_shipping_info,add_shipping_info,session,3034,45362,0.066884,3221,45193,0.071272,6.560481,-2.603571,0.009226,True


## Пояснення роботи коду:

Код написаний на мові Python і використовує бібліотеки `pandas`, `statsmodels` та `numpy` для аналізу результатів A/B-тестування.

### 1. Імпорт бібліотек:

* `import pandas as pd`: Імпортує бібліотеку `pandas` для роботи з табличними даними (DataFrame).
* `from statsmodels.stats.proportion import proportions_ztest`: Імпортує функцію `proportions_ztest` з бібліотеки `statsmodels` для проведення Z-тесту пропорцій, який використовується для визначення статистичної значущості різниці між двома вибірками.
* `import numpy as np`: Імпортує бібліотеку `numpy` для виконання числових операцій, зокрема для представлення масивів та обробки відсутніх значень (`np.nan`).

### 2. Визначення функції `analyze_ab_test(df)`:

* Ця функція приймає на вхід DataFrame `df`, який містить дані A/B-тестування.
* `results =`: Ініціалізує порожній список `results`, який буде використовуватися для зберігання результатів аналізу для кожної метрики та кожного тесту.
* `unique_tests = df['test'].unique()`: Отримує унікальні значення з колонки `test` DataFrame, щоб ітеруватися по кожному проведеному тесту.
* `unique_metrics = [...]`: Визначає список метрик, які необхідно проаналізувати. Зверніть увагу, що `'session'` включено до цього списку, але обробляється окремо як знаменник.

### 3. Основний цикл аналізу:

* `for test in unique_tests:`: Зовнішній цикл, який ітерується по кожному унікальному значенню тесту.
* `test_df = df[df['test'] == test]`: Фільтрує DataFrame, залишаючи тільки дані для поточного тесту.
* `for metric in unique_metrics:`: Внутрішній цикл, який ітерується по кожній визначеній метриці.
* `if metric == 'session': continue`: Пропускає метрику `'session'` на цій ітерації, оскільки вона буде використовуватися як знаменник для розрахунку conversion rate інших метрик.

### 4. Визначення чисельника та знаменника:

* `numerator_event = metric`: Встановлює поточну метрику як подію-чисельник.
* `denominator_event = 'session'`: Встановлює `'session'` як подію-знаменник для розрахунку conversion rate.

### 5. Аналіз для групи 1:

* `group1_df = test_df[test_df['test_group'] == 1]`: Фільтрує дані для поточної групи (`test_group` = 1).
* `numerator_count_group1 = group1_df[group1_df['event_name'] == numerator_event]['value'].sum()`: Обчислює суму значень у колонці `value` для подій, що відповідають `numerator_event` у групі 1.
* `denominator_count_group1 = group1_df[group1_df['event_name'] == denominator_event]['value'].sum()`: Обчислює суму значень у колонці `value` для подій `'session'` у групі 1.
* `conversion_rate_group1 = (...)`: Обчислює conversion rate для групи 1 як відношення `numerator_count_group1` до `denominator_count_group1`. Якщо `denominator_count_group1` дорівнює 0, встановлює conversion rate в 0, щоб уникнути помилки ділення на нуль.

### 6. Аналіз для групи 2:

* Аналогічно до групи 1, обчислює `numerator_count_group2`, `denominator_count_group2` та `conversion_rate_group2` для групи 2 (`test_group` = 2).

### 7. Розрахунок зміни метрики:

* `metric_change = (...)`: Обчислює відсоткову зміну метрики між групою 1 та групою 2. Якщо `conversion_rate_group1` дорівнює 0, встановлює зміну в `np.inf` (нескінченність), оскільки відсоткова зміна не може бути обчислена.

### 8. Статистичний Z-тест:

* `if denominator_count_group1 > 0 and denominator_count_group2 > 0:`: Перевіряє, чи є ненульові значення для знаменників обох груп, щоб уникнути помилки при проведенні тесту.
* `count = np.array([numerator_count_group1, numerator_count_group2])`: Створює масив з кількістю успіхів (кількість подій-чисельників) для обох груп.
* `nobs = np.array([denominator_count_group1, denominator_count_group2])`: Створює масив із загальною кількістю спостережень (кількість сесій) для обох груп.
* `z_stat, p_value = proportions_ztest(count, nobs, alternative='two-sided')`: Виконує Z-тест пропорцій. `alternative='two-sided'` вказує, що ми перевіряємо, чи є статистично значуща різниця між пропорціями обох груп (а не чи одна група краща за іншу).
* `significant = p_value < 0.05`: Визначає, чи є результат статистично значущим на рівні значущості 0.05 (зазвичай використовуваний поріг).
* `else:`: Якщо знаменник хоча б однієї з груп дорівнює 0, встановлює значення `z_stat`, `p_value` та `significant` в `np.nan` та `False` відповідно.

### 9. Збереження результатів:

* `results.append(...)`: Додає словник з результатами аналізу для поточної комбінації тесту та метрики до списку `results`. Словник містить всі необхідні колонки: `'test'`, `'metric'`, `'numerator_event'`, `'denominator_event'`, кількість чисельників та знаменників для обох груп, conversion rate для обох груп, відсоткову зміну метрики, z-статистику, p-значення та ознаку статистичної значущості.

### 10. Повернення DataFrame:

* `return pd.DataFrame(results)`: Після обробки всіх тестів та метрик, функція повертає DataFrame, створений зі списку `results`.

### 11. Застосування функції та вивід результату:

* `pivot_table = analyze_ab_test(df)`: Викликає функцію `analyze_ab_test` з вашим DataFrame `df` та зберігає результат у змінній `pivot_table`.
* `pivot_table`: Відображає отриманий DataFrame зі статистикою A/B-тестування.

Цей код ефективно обчислює всі необхідні метрики та проводить базовий статистичний аналіз для порівняння результатів A/B-тестування між двома групами користувачів.

In [ ]:
from google.colab import files

pivot_table.to_csv('ab_test_results.csv', index=False)
files.download('ab_test_results.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### [ab_test_results.csv](https://drive.google.com/file/d/1yqe9xF9JQNOcZlhPJAt5EZ2HQ_SD8PKa/view?usp=sharing)

## [A/B Test Analysis Dashboard](https://public.tableau.com/app/profile/denys.khvashchenko/viz/ABTestAnalysis_17430179599570/ABTestAnalysis?publish=yes)